In [54]:
import pandas as pd
import numpy as np
import dill as pickle

import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm


from statistics import mean, stdev

import seaborn as sns

import spacy
from unicodedata import normalize

from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation,TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler,FunctionTransformer
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import VotingClassifier, RandomForestClassifier


import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings(action = 'once')


In [28]:
# Load the dataset
df = pd.read_csv("./data/train.csv")
# full_training = pd.read_csv("./data/train.csv")
# df = pd.DataFrame(full_training.sample(500000))

#df= df_full.sample(100000)


/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
# This is where you can do all your processing
print("Joining and Removing Features")
df['Helpfulness'] = df['HelpfulnessNumerator'] / df['HelpfulnessDenominator']
df['Helpfulness'] = df['Helpfulness'].fillna(0)

df['Text'] = df['Summary']+ ' ' +df['Text'] # combine text fields 
df['Raw_text'] = df['Text']
df['Review_len'] = df["Text"].str.len() # review length
df.drop('HelpfulnessDenominator', axis=1, inplace=True) # remove denominator field
df.drop('HelpfulnessNumerator', axis=1, inplace=True) # remove nominator field
df.drop('Summary', axis=1, inplace=True) # remove summary 
# df = df.dropna(subset=['Text']) # remove any empty text fields 
df.Text = df.Text.fillna('fill') # instead of getting rid of no text fields, fill with word "fill"
df.Raw_text = df.Raw_text.fillna('fill') # instead of getting rid of no text fields, fill with word "fill"

Joining and Removing Features


In [30]:
# Add time review was posted 
# Month may not add much infor

df['Date'] = pd.to_datetime(df['Time'], unit='s')
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

df.drop('Date', axis=1, inplace=True) # remove date field
df.drop('Time', axis=1, inplace=True) # remove Time field
# df.drop('Month', axis=1, inplace=True) # remove Month field

In [31]:
print("Binning Years")
df['Year_bin'] = df['Year'].astype(int)
df.loc[ df['Year'] <= 1998, 'Year_bin'] = 0 #0
df.loc[(df['Year'] > 1998) & (df['Year'] <= 2000), 'Year_bin'] = 1 # 0
df.loc[(df['Year'] > 2000) & (df['Year'] <= 2002), 'Year_bin'] = 2 # 1
df.loc[(df['Year'] > 2002) & (df['Year'] <= 2004), 'Year_bin'] = 3 # 1
df.loc[(df['Year'] > 2004) & (df['Year'] <= 2006), 'Year_bin'] = 4 # 2
df.loc[(df['Year'] > 2006) & (df['Year'] <= 2008), 'Year_bin'] = 5 # 2
df.loc[(df['Year'] > 2008) & (df['Year'] <= 2010), 'Year_bin'] = 6 # 2
df.loc[(df['Year'] > 2010) & (df['Year'] <= 2012), 'Year_bin'] = 7 # 3
df.loc[(df['Year'] > 2012) & (df['Year'] <= 2014), 'Year_bin'] = 8 # 3

Binning Years


In [32]:
# 0,1,2,3,4,5,6,7,8,9,10,11,12
#
#winter: 0, spring: 0.33, summer: 0.66, fall: 1
print("Binning Months")
df['Season'] = df['Month'].astype(int)
df.loc[(df['Month'] > 11 ) & (df['Month'] <= 2), 'Season'] = 0 # winter
df.loc[(df['Month'] > 2) & (df['Month'] <= 5), 'Season'] = .33 # spring
df.loc[(df['Month'] > 5) & (df['Month'] <= 8), 'Season'] = .66 # summer
df.loc[(df['Month'] > 8) & (df['Month'] <= 11), 'Season'] = 1 # Fall

Binning Months


/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
print("downloading nltk")
# download nltk tools
nltk.download([
"names",
"stopwords",
"averaged_perceptron_tagger",
"wordnet",
'omw-1.4',
"vader_lexicon",
"punkt",])

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


downloading nltk


[nltk_data] Downloading package names to
[nltk_data]     /usr4/ugrad/arnaudh/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /usr4/ugrad/arnaudh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr4/ugrad/arnaudh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /usr4/ugrad/arnaudh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /usr4/ugrad/arnaudh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr4/ugrad/arnaudh/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /usr

True

In [34]:
print("Processing Text Data")
# remove any non A-Z characters in review text
regex = re.compile('[^a-zA-Z]')
df['Text'] = df['Text'].apply(lambda x: regex.sub(" ", x))

Processing Text Data


In [35]:
print("Removing Stopwords")
# function for removing all stopwords from a piece of text 
stopwords=nltk.corpus.stopwords.words("english")
def rem_stopw(words):
    words = [w for w in words if w.lower() not in stopwords]
    return words

#remove stopwords from the text in df 
df['Text'] = df['Text'].apply(lambda x: nltk.word_tokenize(x))
df['Text'] = df['Text'].apply(lambda x: rem_stopw(x)) #remove stopwords

Removing Stopwords


In [36]:
# re-join the tokenized texrt so that VADER can work properly
df['Text'] = df['Text'].apply(lambda x:" ".join(x))

In [37]:
#SIA works on raw text, not anything tokenized, hence the above operation
print("Performing Sentiment Analysis")
sia = SentimentIntensityAnalyzer()
def sentiment(text):
    scores = []
    #all_scores = []
    for sentence in nltk.sent_tokenize(text):
        scores.append(sia.polarity_scores(sentence)["compound"])
        #all_scores.append(sia.polarity_scores(sentence))
    try:
        scores = mean(scores)
    except:
        scores = np.nan
    
    return scores

# add sentiment score to dataframe
df['Sentiment'] = df['Text'].apply(lambda x: sentiment(x))

Performing Sentiment Analysis


In [38]:
# NORMALIZE NUMERICAL DATA
print('Normalizing Numerical Data')
def scale_col(col):
    return (col - col.min()) / (col.max() - col.min())
df['Review_len'] = scale_col(df['Review_len'])
df['Month'] = scale_col(df['Month'])
# df['Season'] = scale_col(df['Season'])
df['Year_bin'] = scale_col(df['Year_bin'])
df['Year'] = scale_col(df['Year'])
df['Sentiment'] = scale_col(df['Sentiment'])

Normalizing Numerical Data


In [39]:
# Load test set
submissionSet = pd.read_csv("./data/test.csv")

# Merge on Id so that the test set can have feature columns as well
testX= pd.merge(df, submissionSet, left_on='Id', right_on='Id')
testX = testX.drop(columns=['Score_x'])
testX = testX.rename(columns={'Score_y': 'Score'})

# The training set is where the score is not null
trainX =  df[df['Score'].notnull()]

testX.to_csv("./data/X_test.csv", index=False)
trainX.to_csv("./data/X_train.csv", index=False)

##############################################################################################

######################################## MODELS ##############################################

In [40]:
trainX['Text']= trainX['Text'].fillna('fill')
trainX['Raw_text']= trainX['Raw_text'].fillna('fill')
trainX['Sentiment']= trainX['Sentiment'].fillna(trainX['Sentiment'].mean())
trainX['Review_len']=trainX['Review_len'].fillna(0)

<ipython-input-40-d02564e73f4b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainX['Text']= trainX['Text'].fillna('fill')


In [41]:
testX['Text'] = testX['Text'].fillna('fill')
testX['Raw_text']= testX['Raw_text'].fillna('fill')
testX['Review_len']=testX['Review_len'].fillna(0)
testX['Sentiment']= testX['Sentiment'].fillna(testX['Sentiment'].mean())

In [42]:
X_submission = testX

In [43]:
X_submission['Text']= X_submission['Text'].fillna('fill')
X_submission['Raw_text']= X_submission['Raw_text'].fillna('fill')
X_submission['Review_len']=X_submission['Review_len'].fillna(0)
X_submission['Sentiment']= X_submission['Sentiment'].fillna(X_submission['Sentiment'].mean())

In [44]:
# Load files into DataFrames
#X_train = pd.read_csv("./data/X_train.csv")

X_train = trainX


# Split training set into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(
        X_train.drop(['Score'], axis=1),
        X_train['Score'],
        test_size=1/4.0,
        random_state=0
    )

# This is where you can do more feature selection
X_train_processed =  X_train.drop(columns=['Id', 'ProductId', 'UserId', 'Text'])
X_test_processed = X_test.drop(columns=['Id', 'ProductId', 'UserId', 'Text'])
X_submission_processed = X_submission.drop(columns=[
    'Id', 'ProductId', 'UserId', 'Score', 'Text'])
# X_submission_processed = X_test_processed

In [45]:
# Apply pipeline union 

In [46]:
X_train_processed.isnull().sum()

Helpfulness    0
Raw_text       0
Review_len     0
Month          0
Year           0
Year_bin       0
Season         0
Sentiment      0
dtype: int64

In [47]:
get_text_data = FunctionTransformer(lambda x: x['Raw_text'], validate=False)
get_numeric_data = FunctionTransformer(lambda x:
                                       x[['Helpfulness',
                                          'Review_len',
                                          'Season',
                                          'Month',
                                          'Year',
                                          'Year_bin',
                                          'Sentiment',
                                         ]], validate=False)

In [48]:
process_and_join_features = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector', get_text_data),
                 ('vec', CountVectorizer(ngram_range=(1,3)))
            ]))
         ])),
    ('clf', OneVsRestClassifier(LogisticRegression()))
])

In [ ]:
_ = process_and_join_features.fit(X_train_processed, Y_train)

pred_joined = process_and_join_features.predict(X_test_processed)

print("LogReg Accuracy: ", np.mean(pred_joined == Y_test))

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

In [56]:
print("LogReg Accuracy: ", np.mean(pred_joined == Y_test))

LogReg Accuracy:  0.6785657042108396


In [53]:
X_submission['Score'] = process_and_join_features.predict(X_submission_processed)
submission = X_submission[['Id', 'Score']]
submission.to_csv("./data/submission.csv", index=False)

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
# Save the model
with open ('pred_LR', 'wb') as files:
    pickle.dump(process_and_join_features, files)

/share/pkg.7/python3/3.8.10/install/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
with open ('pred_LR', 'rb') as f:
    model = pickle.load(f)

In [ ]:
m_pred = model.predict(X_test_processed)

print("LogReg Accuracy: ", np.mean( m_pred == Y_test))